In [1]:
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon

In [2]:
p_taipei = pd.read_csv("points_taipei.csv")
p_taipei['index'] = range(0, len(p_taipei))

p_new_taipei = pd.read_csv("points_new_taipei.csv")
p_new_taipei['index'] = range(0, len(p_new_taipei))

def create_geometry_gdf(df):
    geometry = [Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, x2, y1, y2 in zip(df['x1'], df['x2'], df['y1'], df['y2'])]
    geometry_gdf = gpd.GeoDataFrame(geometry=geometry)
    geometry_gdf['index'] = range(0, len(df))
    return geometry_gdf

geometry_gdf_t = create_geometry_gdf(p_taipei)
geometry_gdf_nt = create_geometry_gdf(p_new_taipei)

In [3]:
def trans_data(file_path):
        df = pd.read_csv(file_path, header=0)
        trans1 = df[df['CATEGORY'].isin(['公車站', '自行車租賃站','捷運設施'])]
        trans2 = df[df['CATEGORY'].isin(['公路設施', '鐵路設施', '轉運站', '客運站'])]
        trans1_100_gdf, trans1_500_gdf, trans1_800_gdf = trans_count1(trans1)
        trans2_500_gdf, trans2_1000_gdf, trans2_2000_gdf = trans_count2(trans2)

        return trans1_100_gdf, trans1_500_gdf, trans1_800_gdf, trans2_500_gdf, trans2_1000_gdf, trans2_2000_gdf
        
def trans_count1(trans):
        # 創建一個包含醫院位置的GeoDataFrame
        trans_points1 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(trans['POI_X'], trans['POI_Y'])])
        
        trans_100 = trans_points1.buffer(100)
        trans_500 = trans_points1.buffer(500)
        trans_800 = trans_points1.buffer(800)

        trans_100_gdf = gpd.GeoDataFrame(geometry=trans_100, crs=trans_points1.crs)
        trans_500_gdf = gpd.GeoDataFrame(geometry=trans_500, crs=trans_points1.crs)
        trans_800_gdf = gpd.GeoDataFrame(geometry=trans_800, crs=trans_points1.crs)

        return trans_100_gdf, trans_500_gdf, trans_800_gdf

def trans_count2(trans):
        # 創建一個包含醫院位置的GeoDataFrame
        trans_points2 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(trans['POI_X'], trans['POI_Y'])])

        trans_500 = trans_points2.buffer(500)
        trans_1000 = trans_points2.buffer(1000)
        trans_2000 = trans_points2.buffer(2000)

        trans_500_gdf = gpd.GeoDataFrame(geometry=trans_500, crs=trans_points2.crs)
        trans_1000_gdf = gpd.GeoDataFrame(geometry=trans_1000, crs=trans_points2.crs)
        trans_2000_gdf = gpd.GeoDataFrame(geometry=trans_2000, crs=trans_points2.crs)

        return trans_500_gdf, trans_1000_gdf, trans_2000_gdf
    

def overlay_and_count(geometry_gdf, trans_gdf, prefix):
    intersection = geometry_gdf.overlay(trans_gdf, how='intersection')
    count_df = intersection.groupby('index').size().reset_index(name=prefix + '_cnt')
    return count_df

def calculate_trans_counts(geometry_gdf, trans1_100_gdf, trans1_500_gdf, trans1_800_gdf, trans2_500_gdf, trans2_1000_gdf, trans2_2000_gdf):
    trans1_100_cnt = overlay_and_count(geometry_gdf, trans1_100_gdf, 'trans1_100')
    trans1_500_cnt = overlay_and_count(geometry_gdf, trans1_500_gdf, 'trans1_500')
    trans1_800_cnt = overlay_and_count(geometry_gdf, trans1_800_gdf, 'trans1_800')
    trans2_500_cnt = overlay_and_count(geometry_gdf, trans2_500_gdf, 'trans2_500')
    trans2_1000_cnt = overlay_and_count(geometry_gdf, trans2_1000_gdf, 'trans2_1000')
    trans2_2000_cnt = overlay_and_count(geometry_gdf, trans2_2000_gdf, 'trans2_2000')
    return trans1_100_cnt, trans1_500_cnt, trans1_800_cnt, trans2_500_cnt, trans2_1000_cnt, trans2_2000_cnt


def calculate_weighted_score(df, *args):
    # 進行多個 merge 操作
    for arg in args:
        df = df.merge(arg, on='index', how='left')
    
    # 將 NaN 值填充為 0
    df = df.fillna(0)
    
    # 計算加權分數
    df['交通加權分數'] = (
        df['trans1_100_cnt'] * 1 +
        df['trans1_500_cnt'] * 0.6 +
        df['trans1_800_cnt'] * 0.3 +
        df['trans2_500_cnt'] * 0.6 +
        df['trans2_1000_cnt'] * 0.2 +
        df['trans2_2000_cnt'] * 0.5
    )
    
    return df


In [4]:
for i in range(2019,2025):
    trans1_100_gdf, trans1_500_gdf, trans1_800_gdf, trans2_500_gdf, trans2_1000_gdf, trans2_2000_gdf = trans_data(f"data_yaer\{i}_transport.csv")
    
    # 呼叫函式
    trans1_100_cnt_t, trans1_500_cnt_t, trans1_800_cnt_t, trans2_500_cnt_t, trans2_1000_cnt_t, trans2_2000_cnt_t = calculate_trans_counts(geometry_gdf_t, trans1_100_gdf, trans1_500_gdf, trans1_800_gdf, trans2_500_gdf, trans2_1000_gdf, trans2_2000_gdf)
    trans1_100_cnt_nt, trans1_500_cnt_nt, trans1_800_cnt_nt, trans2_500_cnt_nt, trans2_1000_cnt_nt, trans2_2000_cnt_nt = calculate_trans_counts(geometry_gdf_nt, trans1_100_gdf, trans1_500_gdf, trans1_800_gdf, trans2_500_gdf, trans2_1000_gdf, trans2_2000_gdf)

    # 呼叫函式
    result_t = calculate_weighted_score(p_taipei, trans1_100_cnt_t, trans1_500_cnt_t, trans1_800_cnt_t, trans2_500_cnt_t, trans2_1000_cnt_t, trans2_2000_cnt_t)
    result_nt = calculate_weighted_score(p_new_taipei, trans1_100_cnt_nt, trans1_500_cnt_nt, trans1_800_cnt_nt, trans2_500_cnt_nt, trans2_1000_cnt_nt, trans2_2000_cnt_nt)
    # 寫入csv
    result_t.to_csv(f"{i}_taipei_trans.csv", index=False)
    result_nt.to_csv(f"{i}_new_taipei_trans.csv", index=False)
    print(i)

2019
2020
2021
2022
2023
2024
